<a href="https://colab.research.google.com/github/bonaguidin/SpaceX-Falcon-9-Project/blob/main/7_SpaceX_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash pandas plotly

In [ ]:
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:

# Read the spacex data into pandas dataframe
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Extract unique launch sites from df
launch_sites = spacex_df['Launch Site'].unique()

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                      style={'textAlign': 'center', 'color': '#503D36',
                                             'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'}
                                            ] + [{'label': site, 'value': site} for site in launch_sites],
                                            value='ALL',
                                            placeholder='Select a Launch Site here',
                                            searchable=True
                                            ),
                                html.Br(),

                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(id='payload-slider',
                                               min=0,
                                               max=10000,
                                               step=1000,
                                               marks={i: str(i) for i in range(0, 10001, 1000)},
                                               value=[min_payload, max_payload]
                                               ),

                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # For all sites, count successes for each site
        success_counts = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
        total_counts = spacex_df.groupby('Launch Site').size().reset_index(name='total')
        success_counts['failure'] = total_counts['total'] - success_counts['class']

        fig = px.pie(success_counts,
                    values='class',
                    names='Launch Site',
                    title='Total Success Launches By Site')
        return fig
    else:
        # For a specific site, show success vs. failure
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        success_failure_counts = filtered_df['class'].value_counts().reset_index()
        success_failure_counts.columns = ['Outcome', 'Count']
        # Map 1 to Success and 0 to Failure for better readability
        success_failure_counts['Outcome'] = success_failure_counts['Outcome'].map({1: 'Success', 0: 'Failure'})

        fig = px.pie(success_failure_counts,
                    values='Count',
                    names='Outcome',
                    title=f'Success vs Failure Launches for {entered_site}')
        return fig

@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_chart(entered_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(low, high)]

    if entered_site == 'ALL':
        fig = px.scatter(filtered_df,
                         x='Payload Mass (kg)',
                         y='class',
                         color='Booster Version',
                         title='Correlation between Payload Mass and Success for all sites')
    else:
        filtered_site_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(filtered_site_df,
                        x='Payload Mass (kg)',
                        y='class',
                        color='Booster Version',
                        title=f'Correlation between Payload and Success for site {entered_site}')
    return fig

# Run the app
if __name__ == '__main__':
    app.run(jupyter_mode='external')

    ####################################################
    ## OPEN BELOW LINK IN A NEW TAB TO VIEW DASHBOARD ##
    ####################################################